<a href="https://colab.research.google.com/github/dmitriygorlov/Yandex.Practikum_Data_Science/blob/main/Module-01_01-Data-Processing/project-1_borrower-reliability-survey.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Исследование надёжности заёмщиков

Нужно разобраться, влияет ли семейное положение и количество детей клиента на факт погашения кредита в срок. Входные данные от банка — статистика о платёжеспособности клиентов.

Данные представлены в виде таблицы с информацией по клиентам и наличию или отсутствию у них невыплаченных кредитов. По проекту планируются следующие шаги:

- Шаг 1. Прочтение

- Шаг 2. Предобработка данных

- Шаг 3. Ответы на вопросы

- Шаг 4. Общий вывод


## Шаг 1. Прочтение

In [1]:
import pandas as pd

data = pd.read_csv('https://code.s3.yandex.net/datasets/data.csv')

data.info()

print(data.head(20))


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 21525 entries, 0 to 21524
Data columns (total 12 columns):
 #   Column            Non-Null Count  Dtype  
---  ------            --------------  -----  
 0   children          21525 non-null  int64  
 1   days_employed     19351 non-null  float64
 2   dob_years         21525 non-null  int64  
 3   education         21525 non-null  object 
 4   education_id      21525 non-null  int64  
 5   family_status     21525 non-null  object 
 6   family_status_id  21525 non-null  int64  
 7   gender            21525 non-null  object 
 8   income_type       21525 non-null  object 
 9   debt              21525 non-null  int64  
 10  total_income      19351 non-null  float64
 11  purpose           21525 non-null  object 
dtypes: float64(2), int64(5), object(5)
memory usage: 2.0+ MB
    children  ...                                 purpose
0          1  ...                           покупка жилья
1          1  ...                 приобретение автомоби

**Вывод**

Прочитали файл, запросили инфу и прочитали первые 20 строк. <br>

<i> В наборе есть: <br>
- Логическая переменная 'debt' (либо 0 либо 1) <br>
- количественные (их можно сравнить) переменные ('children', 'days_employed', 'dob_years', 'education_id', 'family_status_id', 'total_income')<br>
- Категориальные ('education', 'family_status', 'gender', 'income_type', 'purpose') <br>
</i>

В столбцах 'days_employed' и 'total_income' данных меньше, чем в других столбцах - видимо пропуски, также они единственные являются числами с плавающей точкой. Надо будет убрать эти пропуски, перевести в int64 для использования. <br>
Есть отрицательные значения стажа - это вероятно ошибка ввода дат и нужно будет перевести в плюсы (модуль?)
В столбце 'education' есть значения прописными и обычными буквами - надо будет унифицировать

## Шаг 2. Предобработка данных 

### Обработка пропусков

In [2]:
data['days_employed'].isnull().sum()
# print (data[data['days_employed'].isnull()].head(20))

def plusik(row):
    days_employed = row['days_employed']
    if days_employed < 0: 
        vozvrat = -row['days_employed']
        return vozvrat
    if days_employed >= 0:
        vozvrat = row['days_employed']
        return vozvrat

    
data['days_employed'] = data.apply(plusik, axis=1)

data_null = data[data['total_income'].isna()] 

# добавил создание "нулевой" таблицы для простоты дальнейших сравнений на случайность


# сверка "глазами" на случайность
# print(data.head(20))
# print(data_null.head(20))



# сверка группировкой на случайность по категориальной переменной 'family_status'
print(data_null.groupby('family_status').agg({'family_status': ['count']}))
print(data.groupby('family_status').agg({'family_status': ['count']}))

# создание сводных таблиц
data_pivot = data.pivot_table(index='family_status', values='dob_years', aggfunc=['count', 'mean'])
data_null_pivot = data_null.pivot_table(index='family_status', values='dob_years', aggfunc=['count', 'mean'])

# проверка на случайность по количественной переменной 'dob_years'
print(data_pivot)
print(data_null_pivot)


average_days_employed = data['days_employed'].median()
average_total_income = data['total_income'].median()

data['days_employed'].fillna(average_days_employed, inplace=True)
data['total_income'].fillna(average_total_income, inplace=True)

    
# print(data.head(15))


                      family_status
                              count
family_status                      
Не женат / не замужем           288
в разводе                       112
вдовец / вдова                   95
гражданский брак                442
женат / замужем                1237
                      family_status
                              count
family_status                      
Не женат / не замужем          2813
в разводе                      1195
вдовец / вдова                  960
гражданский брак               4177
женат / замужем               12380
                          count       mean
                      dob_years  dob_years
family_status                             
Не женат / не замужем      2813  38.369357
в разводе                  1195  45.517992
вдовец / вдова              960  56.513542
гражданский брак           4177  42.130476
женат / замужем           12380  43.564701
                          count       mean
                      dob_years  dob_

**Вывод**

Смотрим сколько пропущено - это 2174 значения <br>
Проверяем насколько совпадают пустые значения в столбце значения 'days_employed' и остальными. Точное совпадение с 'total_income', у остальных такого совпадения не замечено. Пустые данные появились скорее всего из-за отсутствия данных о трудоустройстве при анкетировании, зависимостей больше нетДля подтвержения печатаем датафрейм с фильтрацией по пустым значениям с пустыми значениями<br>
Нужно заменить пропуски средним значением для удобства подсчёта (до этого в 'days_employed' переделать отрицательные значения.
Пишем функцию 'plusik'по переводу в положительное значение и применяем её через apply.<br>
Теперь данные в обоих столбцах готовы к замене среднего (берём медиану как более "честное" среднее в сильно разросанных данных), заменяем значения медианой в 'days_employed' и 'total_income'


### Замена типа данных

In [3]:
# data.info()

data['days_employed'] = data['days_employed'].astype('int')
data['total_income'] = data['total_income'].astype('int')

# data.info()



**Вывод**

Для унификации и успешной дальнейшей обработки данных на корреляцию, нужно заменить 2 столца формата 'float64' на 'int64', меняем через astype.

### Обработка дубликатов

In [4]:
# print(data.head(15))

data['education'] = data['education'].str.lower()
# print(data.head(15))

# data['education'].value_counts()

# data['purpose'].value_counts()

# data['family_status'].value_counts()
data['family_status'] = data['family_status'].str.lower()
# data['family_status'].value_counts()


# data['income_type'].value_counts()

# data['gender'].value_counts()
data = data[data['gender'] != 'XNA']
# data['gender'].value_counts()

# data.duplicated().sum()
# print(data[data.duplicated() == True].head(20))
data = data.drop_duplicates().reset_index(drop = True)
# data.duplicated().sum()


**Вывод**

Из пробных таблиц мы знаем о разных регистрах в столбце 'education'. Меняем в нем все значения на нижний регистр с помощью '.str.lower'. Проверим его и все остальные столбцы 'object' с помощью value_counts
Для стандартизации пройдем обработкой и по family_status, а в столбце gender удалим неподходящее значение 'XNA'
Посчитаем дубликаты в таблице и выведем 20 строк. Исходя из полученного повторы - это те элементы, где не было информации про трудоустройство Это значит, что мы спокойно удаляем повторы через drop_dublicates и обновить индексы

### Лемматизация

In [5]:
diction = {'a': 1, 'b': 2}
diction.items()

dict_items([('a', 1), ('b', 2)])

In [6]:
from pymystem3 import Mystem
m = Mystem()
from collections import Counter

In [7]:
# Это специально. чтобы colab не вис на mystem 
!wget http://download.cdn.yandex.net/mystem/mystem-3.0-linux3.1-64bit.tar.gz
!tar -xvf mystem-3.0-linux3.1-64bit.tar.gz
!cp mystem /root/.local/bin/mystem

--2021-09-05 15:27:04--  http://download.cdn.yandex.net/mystem/mystem-3.0-linux3.1-64bit.tar.gz
Resolving download.cdn.yandex.net (download.cdn.yandex.net)... 5.45.205.244, 5.45.205.243, 5.45.205.242, ...
Connecting to download.cdn.yandex.net (download.cdn.yandex.net)|5.45.205.244|:80... connected.
HTTP request sent, awaiting response... 302 Found
Location: https://cachev2-mskm901.cdn.yandex.net/download.cdn.yandex.net/mystem/mystem-3.0-linux3.1-64bit.tar.gz?lid=235 [following]
--2021-09-05 15:27:04--  https://cachev2-mskm901.cdn.yandex.net/download.cdn.yandex.net/mystem/mystem-3.0-linux3.1-64bit.tar.gz?lid=235
Resolving cachev2-mskm901.cdn.yandex.net (cachev2-mskm901.cdn.yandex.net)... 5.45.220.90, 2a02:6b8:0:2002::990
Connecting to cachev2-mskm901.cdn.yandex.net (cachev2-mskm901.cdn.yandex.net)|5.45.220.90|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 16457938 (16M) [application/octet-stream]
Saving to: ‘mystem-3.0-linux3.1-64bit.tar.gz.2’

mystem-3.0-linu

In [8]:
data['purpose'].value_counts()

purposes = []
for element in data['purpose']:
    lemma = m.lemmatize(element)
    purposes.extend(lemma)

unique_purposes = Counter(purposes)
# print(unique_purposes)    

list_unique = []
for elements in unique_purposes:
    if len(elements) >= 5 :
        list_unique.append(elements) 
        

# print(list_unique)

list_uni = {'жилье' : 1, 'автомобиль' : 2, 'образование' : 3, 'свадьба' : 4, 'недвижимость' : 5}

# list_uni.items()

def pyrpose_gr(row):
    pyrpose = row['purpose']
    ret_pyr = m.lemmatize(pyrpose)
    for lemmam in list_uni:
        if lemmam in ret_pyr:
            return lemmam
    return 'другое'

data['pyrpose_group'] = data.apply(pyrpose_gr, axis = 1)

data['pyrpose_group'].value_counts()


недвижимость    6350
жилье           4460
автомобиль      4306
образование     4013
свадьба         2324
Name: pyrpose_group, dtype: int64

**Вывод**

Ранее мы получили информацию, что в столбце 'purpose' есть много значей одинаковых по смыслу, но разных по написанию. Приведем данные к корректным с помощью лемматизации из pymystem3. <br>
Вначале посмотрим на сами значения с помощью value_counts, снова видим, что значений много, но среди них есть схожие группы. В цикле лемматизируем столбец и считаем с помощью Counter наиболее часто встречающиеся варианты. Чтобы убрать пробелы и предлоги фильтруем значения только больше или равно 5 <br>
Из получившегося списка и предыдущего выбираем для словаря 5 значений: 'жилье', 'автомобиль', 'образование', 'свадьба', 'недвижимость'. Создаем функцию pyrpose_gr, которая проходит по столбцу и возвращает что-то из азбуки или 'другое' Примянем функцию и поверяем с value_counts исчерпываемость группировки (наличие 'другое'). Считаем лемматизацию успешной. <br>

### Категоризация данных

In [9]:
# data.info()

def children_type_f(row):
    total = row['children']
    if total == 0:
        return 'нет детей'
    if total > 0:
        return 'дети есть'
    return 'ошибка'
data['children_type'] = data.apply(children_type_f, axis = 1)

# data['children_type'].value_counts()
data = data[data['children_type'] != 'ошибка']
# data['children_type'].value_counts()

data_child = data.groupby('children_type').agg({'debt': ['count', 'mean']})
diction_chil = data['children_type'].unique()
diction_chil2 = data[['family_status']].drop_duplicates().reset_index(drop=True)

# print(diction_chil)
# print(diction_chil2)


data_family_status = data.groupby('family_status').agg({'debt': ['count', 'mean']})
diction_fam_stat = data['family_status'].unique()
diction_fam_stat2 = data[['family_status']].drop_duplicates().reset_index(drop=True)

# print(diction_fam_stat)
# print(diction_fam_stat2)

total_income_mean = data['total_income'].mean()

def total_incom_type_f(row):
    total = row['total_income']
    if total < total_income_mean:
        return 'low'
    if total >= total_income_mean:
        return 'high'
    return 'ошибка'

data['total_incom_type'] = data.apply(total_incom_type_f, axis=1)

# data['total_incom_type'].value_counts()

data_total_incom_type = data.groupby('total_incom_type').agg({'debt': ['count', 'mean']})
data_income_type = data.groupby('income_type').agg({'debt': ['count', 'mean'], 'total_income' : ['mean']})

data_pyrpose_group = data.groupby('pyrpose_group').agg({'debt': ['count', 'mean']})



data_education = data.groupby('education').agg({'debt': ['count', 'mean']})
diction_edu = data['education'].unique()
diction_edu2 = data[['education']].drop_duplicates().reset_index(drop=True)

# print(diction_edu)
# print(diction_edu2)



**Вывод**

Категоризируем данные для разных вопросов, собираем информацию по столбцам через groupby и agg. Нужные нам значения будут по debt - это количество (для понимания выборки) и среднее (для оценивания есть корреляция или нет). <br>
Столбец по детям представляет собой числа, поэтому создадим новый столбец в str по есть/нет детей, Проверяем значение, замечаем ошибочные. Фильтруем таблицу без этих данных. Далее группируем по новому столбцу.<br>
Столбец по семейному статусу тоже собирается просто<br>
Для столбца по типам доходов сделаем числовой срез от среднего (пишем функцию и делаем новый столбец со значениями 'low' и 'high', проверяем и потом группируем. Также сгруппируем и по типу дохода и выведем там средний доход на группу.<br>
Группировка по целям кредита также легко строится.

## Шаг 3. Ответы на вопросы

- Есть ли зависимость между наличием детей и возвратом кредита в срок?

In [10]:
print(data_child)

                debt          
               count      mean
children_type                 
дети есть       7316  0.092537
нет детей      14090  0.075444


**Вывод**

Печатаем и смотрим на столбец mean(для простоты будем говорить о процентах, не сумел поставить процент) и count. 
Размер выборок большой, можно им верить. 
Среди тех, у кого нет детей, процент имеющих задолженность по кредиту ниже. Зависимость есть!

- Есть ли зависимость между семейным положением и возвратом кредита в срок?

In [11]:
print(data_family_status)

                        debt          
                       count      mean
family_status                         
в разводе               1191  0.071369
вдовец / вдова           955  0.065969
гражданский брак        4145  0.093607
женат / замужем        12310  0.075548
не женат / не замужем   2805  0.097683


**Вывод**

Печатаем и смотрим на столбец mean(для простоты будем говорить о процентах, не сумел поставить процент) и count. 
Размер выборок большой, можно им верить. 
Группа вдовцов и вдов имеют наименьший проент задолженности по кредита, также низкие показатели у в разводе и женаты/замужем. У тех, кто в гражданском браке и неженаты/незамужем процент выше.  Зависимость есть!

- Есть ли зависимость между уровнем дохода и возвратом кредита в срок?

In [12]:
print(data_total_incom_type)
print()
print(data_income_type)
print()
data_pivot_incom = data[(data['income_type'] != 'предприниматель')&(data['income_type'] != 'в декрете')&(data['income_type'] != 'безработный')&(data['income_type'] != 'студент')].pivot_table(index='income_type', columns='total_incom_type', values='debt', aggfunc=['count', 'mean'])
print(data_pivot_incom)


                   debt          
                  count      mean
total_incom_type                 
high               7663  0.077515
low               13743  0.083388

                  debt             total_income
                 count      mean           mean
income_type                                    
безработный          2  0.500000  131339.000000
в декрете            1  1.000000   53829.000000
госслужащий       1453  0.059188  168235.437027
компаньон         5068  0.074191  196810.030584
пенсионер         3821  0.056530  137964.716043
предприниматель      2  0.000000  322090.000000
сотрудник        11058  0.095858  159815.493489
студент              1  0.000000   98201.000000

                 count            mean          
total_incom_type  high   low      high       low
income_type                                     
госслужащий        555   898  0.054054  0.062361
компаньон         2456  2612  0.071661  0.076570
пенсионер          918  2903  0.058824  0.055804
сотруд

**Вывод**

Печатаем и смотрим на столбец mean(для простоты будем говорить о процентах, не сумел поставить процент) и count. 
Размер выборок большой в меньше и больше среднего, а также в типах 'Госслужащий', 'Компаньон', 'Пенсионер' и 'Cотрудник'. Группы 'Безработный', 'В декрете', 'Предприниматель', 'Студент' обладают лишь по паре значений - игнорируем их. 

1)При сравнении относительно средней заметно, что процент, имевших задолженность) чуть выше у людей с меньшим заработком. Зависимость есть!
2)При делении на типы дохода заметно меньше процент у Пенсионеров и Госслужащих, средний у Компаньнов и высокий у Сотрудников. Есть сильная зависимость!

- Как разные цели кредита влияют на его возврат в срок?

In [13]:
print(data_pyrpose_group)

               debt          
              count      mean
pyrpose_group                
автомобиль     4295  0.093597
жилье          4452  0.069182
недвижимость   6334  0.074834
образование    4003  0.092431
свадьба        2322  0.080103


**Вывод**

Печатаем и смотрим на столбец mean(для простоты будем говорить о процентах, не сумел поставить процент) и count. 
Размер всех выборок большой, можно им верить. 
Заметно ниже процент у тех, кто брал кредит на жильё и недвижимость, средний у "свадебных" и большой среди бравших кредит на Автомобиль и обраование. Зависимость есть!

## Шаг 4. Общий вывод <a id='step4'></a>

Гипотезы о всех факторах влияния подтвердились! **Самый надежный заемщик обладает следующими характеристиками: нет детей, вдовец/вдова или в разводе, пенсионер или госслужащий, с целью кредита "Жильё"**. А *самый ненадежный обладает обратными харатеристиками: есть дети, не женат/не замужем, сотрудник и цель кредита "Автомобиль" или "Образование"*.